In [1]:
import os
import pandas as pd
import numpy as np

# 1) Import, analyze and join census data

#### Start with population data

In [2]:
census_dir = os.getcwd() + '/../census/'

In [3]:
### read in population data
pop_raw = pd.read_csv(census_dir + 'population.csv', skiprows=1)

### drop unneeded columns and rename main col
pop_df = pop_raw.drop(['Id', 'Margin of Error; Total', 'Geography'], axis = 1)\
            .rename(columns={'Estimate; Total': 'population',
                            'Id2': 'GEOID'})

In [4]:
pop_df.head(1)

,GEOID,population
0,55079000101,4469


#### Read in tenure (homeownership) data

In [5]:
tenure_raw = pd.read_csv(census_dir + 'tenure.csv', skiprows=1)

In [6]:
### drop unneeded columns
tenure_raw = tenure_raw.drop(['Id', 'Margin of Error; Total:', 'Geography',
                              'Margin of Error; Total: - Owner occupied',
                              'Margin of Error; Total: - Renter occupied'], axis = 1)

### rename cols to be more concise
tenure = tenure_raw.rename(columns={'Id2': 'GEOID',
                                    'Estimate; Total:': 'households',
                                   'Estimate; Total: - Owner occupied': 'owner_occupied',
                                   'Estimate; Total: - Renter occupied': 'renter_occupied'})

In [7]:
### calc homeownership rate
tenure['pct_owned'] = np.round(tenure.owner_occupied / tenure.households * 100, 1)

### save to final, simplified df
tenure_final = tenure[['GEOID', 'pct_owned']]

#### Median household income data

In [8]:
income_raw = pd.read_csv(census_dir + 'median_household_income.csv', skiprows=1)

In [9]:
income_raw.head(1)

,Id,Id2,Geography,Estimate; Median household income in the past 12 months (in 2017 inflation-adjusted dollars),Margin of Error; Median household income in the past 12 months (in 2017 inflation-adjusted dollars)
0,1400000US55079000101,55079000101,"Census Tract 1.01, Milwaukee County, Wisconsin",30676,9062


In [10]:
### remove unneeded columns and rename primary one
income_df = income_raw.drop(['Id', 'Geography',
    'Margin of Error; Median household income in the past 12 months (in 2017 inflation-adjusted dollars)'],
            axis = 1)

income_df = income_df.rename(columns={
    'Estimate; Median household income in the past 12 months (in 2017 inflation-adjusted dollars)':\
    'median_hh_income',
    'Id2': 'GEOID'})

In [11]:
#### looks like income field is not numeric. 
income_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 298 entries, 0 to 297
Data columns (total 2 columns):
GEOID               298 non-null int64
median_hh_income    298 non-null object
dtypes: int64(1), object(1)
memory usage: 4.7+ KB


In [12]:
### convert income field to int. replace any fields that are empty with nulls, and copy to new df
income_df.median_hh_income = income_df.median_hh_income.replace('-', np.nan).astype('float')

income_final = income_df.copy(deep=True)

In [13]:
### three tracts have no income available. only one actually has people living in it.
### we'll imput data for that tract with igures from an adjacent one (acknowledge this
### an imperfect approach, but, since a tract's income is more likely to be similar to an
### adjacent tract than the overall city average, I'd argue this is a better way to impute
### a missing value)
income_final[income_final.median_hh_income.isnull()].merge(pop_df, on='GEOID', how='inner')

,GEOID,median_hh_income,population
0,55079009700,NaN,1661
1,55079980000,NaN,0
2,55079990000,NaN,0


In [14]:
### impute missing income with values from adjacent tract
### 55079009700 will take value from 55079012200
income_final.median_hh_income = np.where(income_final.GEOID==55079009700,
                    income_final[income_final.GEOID==55079012200].median_hh_income,
                    income_final.median_hh_income)

In [15]:
### did that work? Yes
income_final[income_final.GEOID==55079009700]

,GEOID,median_hh_income
100,55079009700,23722.0


#### Race/ethnicity data

In [16]:
demos_raw = pd.read_csv(census_dir + 'race_ethnicity.csv', skiprows=1)

In [17]:
demos_raw.head(1)

,Id,Id2,Geography,Estimate; Total:,Margin of Error; Total:,Estimate; Not Hispanic or Latino:,Margin of Error; Not Hispanic or Latino:,Estimate; Not Hispanic or Latino: - White alone,Margin of Error; Not Hispanic or Latino: - White alone,Estimate; Not Hispanic or Latino: - Black or African American alone,...,Estimate; Hispanic or Latino: - Native Hawaiian and Other Pacific Islander alone,Margin of Error; Hispanic or Latino: - Native Hawaiian and Other Pacific Islander alone,Estimate; Hispanic or Latino: - Some other race alone,Margin of Error; Hispanic or Latino: - Some other race alone,Estimate; Hispanic or Latino: - Two or more races:,Margin of Error; Hispanic or Latino: - Two or more races:,Estimate; Hispanic or Latino: - Two or more races: - Two races including Some other race,Margin of Error; Hispanic or Latino: - Two or more races: - Two races including Some other race,"Estimate; Hispanic or Latino: - Two or more races: - Two races excluding Some other race, and three or more races","Margin of Error; Hispanic or Latino: - Two or more races: - Two races excluding Some other race, and three or more races"
0,1400000US55079000101,55079000101,"Census Tract 1.01, Milwaukee County, Wisconsin",4469,453,4312,488,1332,295,2845,...,0,9,95,80,0,9,0,9,0,9


In [18]:
#### need to remove unneeded columns
margin_cols = [x for x in demos_raw.columns if 'Margin' in x]

demos_raw = demos_raw.drop(margin_cols, axis = 1)

In [19]:
### the columns eventually needed are
# white, non-hispanic pct
# hispanic pct
# black, non-hispanic pct
# asian, non-hispanic pct

demos_raw = demos_raw.rename(columns={'Id2': 'GEOID',
    'Estimate; Not Hispanic or Latino: - White alone': 'white_non_hisp',
    'Estimate; Not Hispanic or Latino: - Black or African American alone': 'black_non_hisp',
    'Estimate; Not Hispanic or Latino: - Asian alone': 'asian_non_hisp',
    'Estimate; Hispanic or Latino:': 'hisp_latino'
})

In [20]:
### narrow to just those cosl
demos = demos_raw[['GEOID', 'white_non_hisp', 'black_non_hisp', 'asian_non_hisp', 'hisp_latino']]

In [21]:
### merge with pop data to be able to calculate percentages
#### merge demos with pop
demos_final = pop_df.merge(demos, how='outer', on='GEOID')

demos_final['pct_white'] = np.round(demos_final.white_non_hisp /\
                                    demos_final.population * 100, 1)

demos_final['pct_black'] = np.round(demos_final.black_non_hisp /\
                                    demos_final.population * 100, 1)

demos_final['pct_hisp'] = np.round(demos_final.hisp_latino /\
                                    demos_final.population * 100, 1)

demos_final['pct_asian'] = np.round(demos_final.asian_non_hisp /\
                                    demos_final.population * 100, 1)

In [22]:
### narrow to just pct cols
demos_final = demos_final[['GEOID', 'population', 'pct_white', 'pct_black',
                          'pct_hisp', 'pct_asian']]

#### educational attainment

In [23]:
education_raw = pd.read_csv(census_dir + 'educational_attainment_25_up.csv', skiprows=1)
education_raw.head(1)

,Id,Id2,Geography,Estimate; Total:,Margin of Error; Total:,Estimate; Total: - No schooling completed,Margin of Error; Total: - No schooling completed,Estimate; Total: - Nursery school,Margin of Error; Total: - Nursery school,Estimate; Total: - Kindergarten,...,Estimate; Total: - Associate's degree,Margin of Error; Total: - Associate's degree,Estimate; Total: - Bachelor's degree,Margin of Error; Total: - Bachelor's degree,Estimate; Total: - Master's degree,Margin of Error; Total: - Master's degree,Estimate; Total: - Professional school degree,Margin of Error; Total: - Professional school degree,Estimate; Total: - Doctorate degree,Margin of Error; Total: - Doctorate degree
0,1400000US55079000101,55079000101,"Census Tract 1.01, Milwaukee County, Wisconsin",2581,295,23,28,0,9,0,...,215,136,635,197,158,92,44,56,10,17


In [24]:
### rename cols to be more concise
education_raw = education_raw.rename(columns={'Id2': 'GEOID',
                            'Estimate; Total:': 'population_25+',
                            'Estimate; Total: - Bachelor\'s degree': 'bach_degree',
                            'Estimate; Total: - Master\'s degree': 'mast_degree',
                            'Estimate; Total: - Professional school degree': 'prof_degree',
                            'Estimate; Total: - Doctorate degree': 'doc_degree'})

In [25]:
### drop unneeded cols
education_df = education_raw.drop([x for x in education_raw.columns\
                                if 'Estimate' in x or 'Margin' in x], axis = 1)\
                            .drop(['Id', 'Geography'], axis = 1)

In [26]:
### calcualte # of people with bachelor's degree or above
education_df['bach_plus'] = education_df.bach_degree + education_df.mast_degree\
                            + education_df.prof_degree + education_df.doc_degree
    
education_df = education_df.drop(['bach_degree', 'mast_degree',
                                 'prof_degree', 'doc_degree'], axis = 1)

In [27]:
### calc pct of people with bachelor's degree or above
education_df['pct_coll_plus'] = np.round(education_df.bach_plus /\
                                        education_df['population_25+'] * 100, 1)

### narrow to only needed cols
education_final = education_df[['GEOID', 'pct_coll_plus']]

### merge all the datasets

In [28]:
all_census = tenure_final.merge(income_final, how = 'outer', on='GEOID')\
                    .merge(demos_final, how = 'outer', on='GEOID')\
                    .merge(education_final, how = 'outer', on='GEOID')

In [29]:
all_census.head()

,GEOID,pct_owned,median_hh_income,population,pct_white,pct_black,pct_hisp,pct_asian,pct_coll_plus
0,55079000101,23.4,30676.0,4469,29.8,63.7,3.5,1.0,32.8
1,55079000102,46.5,40242.0,3666,26.8,56.0,6.2,9.4,20.3
2,55079000201,17.5,26596.0,5073,14.7,60.4,18.6,2.9,13.3
3,55079000202,53.5,59136.0,6757,39.1,47.0,7.4,2.4,29.2
4,55079000301,90.4,67031.0,1462,83.2,13.2,1.8,1.8,28.6


#### last census step -- import geo data to calculate pop density

In [30]:
### this csv was generated from a tract shapefile in QGIS
tracts = pd.read_csv(census_dir + 'mke_county_tract_geo.csv')

### narrow to area and geoid fields
tracts = tracts[['GEOID', 'ALAND']]

In [31]:
### merge with rest of census data and calc pop density
all_census = all_census.merge(tracts, on='GEOID', how='left')

In [32]:
### calc pop density
all_census['pop_density'] = np.round(all_census.population / all_census.ALAND * 10000, 3)

# 2) Process crime data

In [33]:
crime_dir = os.getcwd() + '/../crime/'

crime_raw = pd.read_csv(crime_dir + 'wibr.csv', encoding='latin1')

In [34]:
### From this data, we're going to calculate homicide rates, other violent crime rates
### and property crime rates, we'll also calculate violent crime trend

### preview fields
crime_raw.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 686232 entries, 0 to 686231
Data columns (total 24 columns):
IncidentNum         686232 non-null int64
ReportedDateTime    686232 non-null object
ReportedYear        686232 non-null int64
ReportedMonth       686232 non-null int64
Location            673358 non-null object
WeaponUsed          332398 non-null object
ALD                 672509 non-null float64
NSP                 416772 non-null float64
POLICE              672509 non-null float64
TRACT               672509 non-null float64
WARD                672509 non-null float64
ZIP                 672509 non-null float64
RoughX              672509 non-null float64
RoughY              672509 non-null float64
Arson               686232 non-null int64
AssaultOffense      686232 non-null int64
Burglary            686232 non-null int64
CriminalDamage      686232 non-null int64
Homicide            686232 non-null int64
LockedVehicle       686232 non-null int64
Robbery             686232 non

In [35]:
### limit data to just crime from 2015-2018
crime_since_15 = crime_raw[(crime_raw.ReportedYear >= 2015)
                          & (crime_raw.ReportedYear <= 2018)].copy(deep=True)

In [36]:
### group crime at the census tract level
crime_by_tract = crime_since_15.groupby('TRACT')['Arson', 'AssaultOffense','Burglary',
                        'CriminalDamage', 'Homicide', 'LockedVehicle', 'Robbery',
                        'SexOffense', 'Theft', 'VehicleTheft'].sum().reset_index()

In [37]:
### group by homicides, other violent crime and property crime
crime_by_tract['other_violent_crime'] = crime_by_tract.AssaultOffense\
                + crime_by_tract.SexOffense + crime_by_tract.Robbery

crime_by_tract['property_crime'] = crime_by_tract.Arson + crime_by_tract.Burglary +\
                crime_by_tract.CriminalDamage + crime_by_tract.LockedVehicle +\
                crime_by_tract.Theft + crime_by_tract.VehicleTheft

In [38]:
### narrow to final fields
crime_by_tract = crime_by_tract[['TRACT', 'Homicide', 'other_violent_crime', 'property_crime']]

In [39]:
### lastly, we want to calculate violent crime trend (including homicides)
### We're going to compare 15/16 to 17/18

### create a violent_crime field
crime_since_15['violent_crime'] = crime_since_15.Homicide + crime_since_15.AssaultOffense\
                            + crime_since_15.SexOffense + crime_since_15.Robbery

crime_15_16 = crime_since_15[crime_since_15.ReportedYear <= 2016]\
                    .groupby('TRACT')['violent_crime'].sum().reset_index()\
                    .rename(columns={'violent_crime': '15_16_violent_crime'})

crime_17_18 = crime_since_15[crime_since_15.ReportedYear >= 2017]\
                    .groupby('TRACT')['violent_crime'].sum().reset_index()\
                    .rename(columns={'violent_crime': '17_18_violent_crime'})

In [40]:
### because percentage changes with small numbers can be misleading, we're just going to use the raw change
crime_changes = crime_15_16.merge(crime_17_18, how='left', on='TRACT').fillna(0)

crime_changes['violent_crime_trend'] = crime_changes['17_18_violent_crime']\
                                        - crime_changes['15_16_violent_crime']

### narrow to final field
crime_changes = crime_changes[['TRACT', 'violent_crime_trend']]

In [41]:
### merge back in with rest of crime data

In [42]:
crime_by_tract = crime_by_tract.merge(crime_changes, how='left', on = 'TRACT')

In [43]:
### lastly, add in the full geo prefix to tract labels that's used in census data
crime_by_tract['GEOID'] = ('55079' + crime_by_tract.TRACT.astype('int').astype('str').replace('.',
                                        0).str.zfill(6)).astype('int')

crime_by_tract.head()

,TRACT,Homicide,other_violent_crime,property_crime,violent_crime_trend,GEOID
0,0.0,0,3,25,-1.0,55079000000
1,101.0,4,533,815,87.0,55079000101
2,102.0,0,316,911,30.0,55079000102
3,201.0,4,624,787,102.0,55079000201
4,202.0,2,353,802,33.0,55079000202


In [44]:
### drop original tract field
crime_by_tract = crime_by_tract.drop('TRACT', axis =1)

# 3) Process assessment and vacant property records

In [45]:
prop_dir = file_dir = os.getcwd() + '/../buildings/'

prop_raw = pd.read_csv(file_dir + 'mprop.csv')

In [46]:
prop_raw.columns

Index(['TAXKEY', 'SUB_ACCT', 'YR_ASSMT', 'CHK_DIGIT', 'TAX_RATE_CD',
       'PLAT_PAGE', 'HOUSE_NR_LO', 'HOUSE_NR_HI', 'HOUSE_NR_SFX', 'SDIR',
       'STREET', 'STTYPE', 'C_A_CLASS', 'C_A_SYMBOL', 'C_A_LAND', 'C_A_IMPRV',
       'C_A_TOTAL', 'C_A_EXM_TYPE', 'C_A_EXM_LAND', 'C_A_EXM_IMPRV',
       'C_A_EXM_TOTAL', 'P_A_CLASS', 'P_A_SYMBOL', 'P_A_LAND', 'P_A_IMPRV',
       'P_A_TOTAL', 'P_A_EXM_TYPE', 'P_A_EXM_LAND', 'P_A_EXM_IMPRV',
       'P_A_EXM_TOTAL', 'LAST_VALUE_CHG', 'REASON_FOR_CHG', 'CONVEY_DATE',
       'CONVEY_TYPE', 'CONVEY_FEE', 'CHG_NR', 'DIV_ORG', 'DIV_DROP',
       'OWNER_NAME_1', 'OWNER_NAME_2', 'OWNER_NAME_3', 'OWNER_MAIL_ADDR',
       'OWNER_CITY_STATE', 'OWNER_ZIP', 'LAST_NAME_CHG', 'NEIGHBORHOOD',
       'EXM_ACREAGE', 'EXM_PER_CT_LAND', 'EXM_PER_CT_IMPRV', 'BLDG_TYPE',
       'NR_STORIES', 'BASEMENT', 'ATTIC', 'NR_UNITS', 'BLDG_AREA', 'YR_BUILT',
       'SWIM_POOL', 'FIREPLACE', 'AIR_CONDITIONING', 'NR_ROOMS', 'BEDROOMS',
       'BATHS', 'POWDER_ROOMS', 'PARKING_SP

In [47]:
''' the fields we care about in this data set are
- GEO_TRACT (Equivalent of census tract)
- OWN_OCPD
- C_A_CLASS
- NR_UNITS
- LAND_USE
- PARKING_TYPE
- C_A_TOTAL
- AIR_CONDITIONING
- BEDROOMS
- BATHS
- BLDG_AREA
- LAND_AREA
- LOT_AREA
- '''
prop_raw.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 161647 entries, 0 to 161646
Data columns (total 92 columns):
TAXKEY              161647 non-null int64
SUB_ACCT            0 non-null float64
YR_ASSMT            161262 non-null float64
CHK_DIGIT           0 non-null float64
TAX_RATE_CD         161644 non-null float64
PLAT_PAGE           139956 non-null float64
HOUSE_NR_LO         161647 non-null int64
HOUSE_NR_HI         161647 non-null int64
HOUSE_NR_SFX        5859 non-null object
SDIR                161643 non-null object
STREET              161643 non-null object
STTYPE              161350 non-null object
C_A_CLASS           161260 non-null float64
C_A_SYMBOL          28915 non-null object
C_A_LAND            161647 non-null int64
C_A_IMPRV           161647 non-null int64
C_A_TOTAL           161647 non-null int64
C_A_EXM_TYPE        8968 non-null float64
C_A_EXM_LAND        161647 non-null int64
C_A_EXM_IMPRV       161647 non-null int64
C_A_EXM_TOTAL       161647 non-null int64
P_A

In [48]:
### how many parcels are there
print(len(prop_raw))
### are any missing tracts?
len(prop_raw[prop_raw.GEO_TRACT.isnull()])

161647


1464

In [49]:
### not much we can do about those. just remove those records
prop = prop_raw[prop_raw.GEO_TRACT.notnull()].copy(deep=True)

In [50]:
### let's start previewing the fields we care about
print(prop.OWN_OCPD.unique())

### create cleaner column for owner occupancy
prop['own_occ'] = np.where(prop.OWN_OCPD=='O',1, 0)

[nan 'O']


In [51]:
### we want to ultimately make calculations regarding residential properties 
### and single family residential properties. start by flagging as residential
### C_A_CLASS of 1 is residential. 5 is mercantil apt. 7 is condo
prop['residential'] = np.where((prop.C_A_CLASS==1) |\
                               (prop.C_A_CLASS==5) |\
                               (prop.C_A_CLASS==7), 1, 0)

In [52]:
### create a flag for sinlge fam
prop['single_fam'] = np.where((prop.C_A_CLASS==1)\
                              & (prop.NR_UNITS==1)\
                              & (prop.LAND_USE==8810), 1, 0)

In [53]:
## Is there a garage?
prop['garage'] = np.where(prop.PARKING_TYPE.notnull(), 1, 0)

#### Read in and join vacant property data

In [54]:
vacants = pd.read_csv(file_dir + 'vacants.csv')

### add a vacant flat that will be useful later
vacants['vacant'] = 1
vacants.head()

,RECORDID,RECORDTYPE,STATUS,DATEOPENED,ADDRFULLLINE,BOOK,PARCELNBR,VALUEIMPROVED,vacant
0,VAC-19-00293,Vacant Building,Inspection Due,2019-06-10 00:00:00,"5259 N 45TH ST, MILWAUKEE, WI 532183418",8810.0,1910512000,40000.0,1
1,VAC-19-00296,Vacant Building,Inspection Due,2019-06-10 00:00:00,"2933 N 26TH ST, MILWAUKEE, WI 532061122",8810.0,3102115000,0.0,1
2,VAC-19-00294,Vacant Building,Inspection Due,2019-06-10 00:00:00,"2622 W MEDFORD AV, MILWAUKEE, WI 532061027",8820.0,3251021000,20900.0,1
3,VAC-19-00286,Vacant Building,Inspection Due,2019-06-07 00:00:00,"3360 S 55TH ST, MILWAUKEE, WI 532194434",8810.0,5300808000,84100.0,1
4,VAC-19-00287,Vacant Building,Inspection Due,2019-06-07 00:00:00,"3280 N 37TH ST, MILWAUKEE, WI 532163724",8810.0,2871122000,63400.0,1


In [55]:
### appears all of these have the same status and should be treated as vacant
print(vacants.RECORDTYPE.unique())
vacants.STATUS.unique()

['Vacant Building']


array(['Inspection Due'], dtype=object)

In [56]:
### all we need are the parcel numbers
vacants = vacants[['PARCELNBR', 'vacant']].drop_duplicates()

In [57]:
### merge with property data
prop = prop.merge(vacants, how='left', left_on='TAXKEY', right_on='PARCELNBR')

In [58]:
### fill non-vacant properties with zeroes
prop.vacant = prop.vacant.fillna(0)

In [59]:
### add the full census identifier for mke county to the tract codes
prop['GEOID'] = ('55079' + prop.GEO_TRACT.astype('int').astype('str').replace('.',
                                        0).str.zfill(6)).astype('int')

In [60]:
### limit data to just single family residential
single_fam = prop[prop.single_fam==1]

In [61]:
#### Group data points and make calculations at the GEOID level

In [62]:
## median val
median_val = single_fam.groupby('GEOID').C_A_TOTAL.median().reset_index()\
                                .rename(columns={'C_A_TOTAL':'median_value'})

In [63]:
## owner occupied rate
tenure = single_fam.groupby('GEOID').own_occ.mean().reset_index()\
                                .rename(columns={'own_occ': 'pct_owned'})

In [64]:
## avg. num bedrooms, bathrooms, bldg size, etc
single_means = single_fam.groupby('GEOID')['BEDROOMS', 'BATHS', 'BLDG_AREA',
                        'LOT_AREA', 'AIR_CONDITIONING', 'garage'].mean().reset_index()

In [65]:
### calculate vacancy rates
vacant_by_geo = prop.groupby('GEOID').vacant.mean().reset_index()

In [66]:
### calculate num properties in each tract
num_props = single_fam.groupby('GEOID').TAXKEY.count().reset_index()\
                        .rename(columns={'TAXKEY': 'num_props'})

In [67]:
### What pct of sq lot footage is residential of any type?
pct_residential = (prop[prop.residential==1].groupby('GEOID').BLDG_AREA.sum() /\
                prop.groupby('GEOID').BLDG_AREA.sum()).reset_index()\
                .rename(columns={'BLDG_AREA': 'pct_residential'})

### ad what pct of sq lot footage is single family only?
pct_single_fam = (single_fam.groupby('GEOID').BLDG_AREA.sum() /\
                prop.groupby('GEOID').BLDG_AREA.sum()).reset_index().\
                rename(columns={'BLDG_AREA': 'pct_single_fam'})

In [68]:
### merge the dfs
### num_props, median_val, tenure, single_means, vacant_by_geo, pct_residential, pct_single_fam
prop_geo = num_props.merge(median_val, how='left', on='GEOID')\
                .merge(tenure, how='left', on='GEOID')\
                .merge(single_means, how='left', on='GEOID')\
                .merge(vacant_by_geo, how='left', on='GEOID')\
                .merge(pct_residential, how='left', on='GEOID')\
                .merge(pct_single_fam, how='left', on='GEOID')

# 4) Merge the final datasets

In [69]:
### drop pct owned col since it's in property data
all_census = all_census.drop(['pct_owned'], axis = 1)

In [70]:
all_census.head()

,GEOID,median_hh_income,population,pct_white,pct_black,pct_hisp,pct_asian,pct_coll_plus,ALAND,pop_density
0,55079000101,30676.0,4469,29.8,63.7,3.5,1.0,32.8,3817688,11.706
1,55079000102,40242.0,3666,26.8,56.0,6.2,9.4,20.3,6508343,5.633
2,55079000201,26596.0,5073,14.7,60.4,18.6,2.9,13.3,7910371,6.413
3,55079000202,59136.0,6757,39.1,47.0,7.4,2.4,29.2,7881718,8.573
4,55079000301,67031.0,1462,83.2,13.2,1.8,1.8,28.6,1985210,7.364


In [71]:
### this census data includes mke county, which we're not analyzing. to filter out
### the county, do an inner join with crime data, which is exclusive to the city
census_crime = all_census.merge(crime_by_tract, how='inner', on='GEOID')

In [72]:
census_crime.head()

,GEOID,median_hh_income,population,pct_white,pct_black,pct_hisp,pct_asian,pct_coll_plus,ALAND,pop_density,Homicide,other_violent_crime,property_crime,violent_crime_trend
0,55079000101,30676.0,4469,29.8,63.7,3.5,1.0,32.8,3817688,11.706,4,533,815,87.0
1,55079000102,40242.0,3666,26.8,56.0,6.2,9.4,20.3,6508343,5.633,0,316,911,30.0
2,55079000201,26596.0,5073,14.7,60.4,18.6,2.9,13.3,7910371,6.413,4,624,787,102.0
3,55079000202,59136.0,6757,39.1,47.0,7.4,2.4,29.2,7881718,8.573,2,353,802,33.0
4,55079000301,67031.0,1462,83.2,13.2,1.8,1.8,28.6,1985210,7.364,0,23,98,-9.0


In [73]:
### merge with property df
final_df = prop_geo.merge(census_crime, on='GEOID', how='left')

### remove any rows in areas where people don't really live

In [74]:
### What's the null record? Looks like nobody lives here, so just drop it
final_df[final_df.median_hh_income.isnull()]

,GEOID,num_props,median_value,pct_owned,BEDROOMS,BATHS,BLDG_AREA,LOT_AREA,AIR_CONDITIONING,garage,...,pct_black,pct_hisp,pct_asian,pct_coll_plus,ALAND,pop_density,Homicide,other_violent_crime,property_crime,violent_crime_trend
193,55079185200,1,163600,1.0,3.0,1.0,2928.0,4375.0,1.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [75]:
### drop the null record 
final_df = final_df[final_df.median_hh_income.notnull()]

### convert income to int
final_df.median_hh_income = final_df.median_hh_income.astype('int')

In [76]:
### export the file
final_df.to_csv(file_dir + '/../analysis/data/final_data.csv', index = False)